In [1]:
from parser import bus_data_df, load_data_df, generator_data_df, fixed_shunt_data_df, branch_data_df, trafo_data_df, switched_shunt_data_df
import pandas as pd
import pandapower as pp
import numpy as np
import os

raw_file = '/Users/tomasvazquez/Desktop/Facultad/Tesis/Datos/DORAA_una_carga.raw'

## Cargamos datos del .raw

In [2]:
bus_data = bus_data_df(raw_file)
load_data = load_data_df(raw_file)
generator_data = generator_data_df(raw_file)
fixed_shunt_data = fixed_shunt_data_df(raw_file)
branch_data = branch_data_df(raw_file)
trafo_data = trafo_data_df(raw_file)

In [3]:
uy_net = pp.create_empty_network()
ieee_30 = pp.networks.case_ieee30()

### Cargamos bus data

In [4]:
for bus in bus_data.index:
    pp.create_bus(uy_net, vn_kv=bus_data.loc[bus, 'BASKV'],
                            name = bus,
                            index=bus_data.loc[bus, 'I'],
                            zone = bus_data.loc[bus, 'ZONE'],
                            in_service = bus_data.loc[bus, 'IDE'] != 4,
                            max_vm_pu = bus_data.loc[bus, 'NVHI'],
                            min_vm_pu = bus_data.loc[bus, 'NVLO']
                            )

### Creamos load data

In [5]:
for bus in load_data.index:
    pp.create_load(uy_net, bus=int(bus),
                    p_mw=load_data.loc[bus, 'PL'],
                    q_mvar=load_data.loc[bus, 'QL'],
                    name = None,
                    index=None,
                    in_service = True,
                    controllable=False
                    )

### Creamos fixed shunt data

In [6]:
for idx in fixed_shunt_data.index:
    pp.create_shunt(uy_net, bus=int(fixed_shunt_data.loc[idx, 'I']),
                    q_mvar=fixed_shunt_data.loc[idx, 'BL'],
                    p_mw=fixed_shunt_data.loc[idx, 'GL'],
                    )

### Creamos generator data

In [7]:
previous_bus = None

for bus in generator_data.index:
    if bus != '4000':
        if bus != previous_bus:
            previous_bus = bus
            pp.create.create_gen(uy_net, bus=int(bus),
                            p_mw=generator_data.loc[bus, 'PG'].sum(),
                            vm_pu=generator_data.loc[bus, 'VS'].sum(),
                            name = None,
                            index=None,
                            in_service = True,
                            controllable=True,
                            max_p_mw = generator_data.loc[bus, 'PT'].sum(),
                            min_p_mw = generator_data.loc[bus, 'PB'].sum(),
                            max_q_mvar = generator_data.loc[bus, 'QT'].sum(),
                            min_q_mvar = generator_data.loc[bus, 'QB'].sum(),
                            )
    else:
        if bus != previous_bus:
            previous_bus = bus
            pp.create.create_ext_grid(uy_net, bus=int(bus),
                            vm_pu=generator_data.loc[bus, 'VS'].sum(),
                            name = None,
                            in_service = True,
                            max_p_mw = generator_data.loc[bus, 'PT'].sum(),
                            min_p_mw = generator_data.loc[bus, 'PB'].sum(),
                            max_q_mvar = generator_data.loc[bus, 'QT'].sum(),
                            min_q_mvar = generator_data.loc[bus, 'QB'].sum(),
                            )

### Creamos branch data

In [8]:
for idx in branch_data.index:
    pp.create_line_from_parameters(uy_net, from_bus=int(branch_data.loc[idx, 'I']),
                    to_bus=int(branch_data.loc[idx, 'J']),
                    length_km=1,
                    r_ohm_per_km=branch_data.loc[idx, 'R'],
                    x_ohm_per_km=branch_data.loc[idx, 'X'],
                    c_nf_per_km=branch_data.loc[idx, 'B']/(2*np.pi*50),
                    max_i_ka=float(branch_data.loc[idx, 'RATE1']) / (np.sqrt(3) * uy_net.bus.loc[int(branch_data.loc[idx, 'I'])]['vn_kv']),
                    name=None,
                    index=None,
                    in_service=True,
                    )

### Creando transfomer data

In [9]:
for idx in trafo_data.index:
    pp.create_transformer_from_parameters(uy_net,
                                    hv_bus=int(trafo_data.loc[idx, 'I']),
                                    lv_bus=int(trafo_data.loc[idx, 'J']),
                                    sn_mva=trafo_data.loc[idx, 'SBASE1-2'],
                                    vn_hv_kv = 500,
                                    vn_lv_kv = 150,
                                    vkr_percent = trafo_data.loc[idx, 'R1-2'] * 100,
                                    vk_percent = ((trafo_data.loc[idx, 'R1-2']**2 + trafo_data.loc[idx, 'X1-2']**2)**0.5) * 100,
                                    pfe_kw = 0.0,
                                    i0_percent = 0.0,
                                    in_service = True,
                                    vector_group = trafo_data.loc[idx, 'VECGRP'],
                                   )

### Guardamos la red

In [10]:
pp.to_pickle(uy_net, '/Users/tomasvazquez/Desktop/Facultad/Tesis/Datos/uy_pp_net_v1.p')  # absolute path

### Visualización de la red

In [11]:
uy_net.ext_grid

,name,bus,vm_pu,va_degree,slack_weight,in_service,min_p_mw,max_p_mw,min_q_mvar,max_q_mvar
0,None,4000,6.00943,0.0,1.0,True,350.0,945.0,-350.1,560.0
